This notebook explains and demonstrates the use of the search_phase_space function. All coordinate transformations are dervied from Bovy (2011). [https://github.com/jobovy/stellarkinematics/blob/master/stellarkinematics.pdf]

To convert from ICRS coordinates to rectangular galactic coordinates, the function first fetches the ra and dec of the north galactic pole, $\alpha_{\textrm{NGP}}$ and $\delta_{\textrm{NGP}}$ in ICRS using galpy. These are used to calculate the sine and cosine of the galactic parallactic angle $\phi$ for each star, according to the equations:

$$\cos \phi = \frac{\sin \delta_{\textrm{NGP}} - \sin \delta \sin b}{\cos \delta \cos b}$$

$$\sin \phi = \frac{\sin(\alpha - \alpha_{\textrm{NGP}})\cos \delta_{\textrm{NGP}}}{\cos b}$$

Where $\alpha$ and $\delta$ are the ra and dec of the star whose coordinates are being converted, and $b$ is its galactic latitude. These are then used to calculate the star's proper motion in $b$ and in its galactic longitude $l$ according to the equation:

$$\mu_{l} \cos b = \mu_{\alpha} \cos{\delta} \cos{\phi} + \mu_{\delta} \sin{\phi}$$

$$\mu_{b} = \mu_{\delta} \cos{\phi} - \mu_{\alpha} \cos{\delta} \sin{\phi}$$

Note that in the code, $\textrm{pmra} \equiv \mu_{\alpha} \cos{\delta}$ and $\textrm{pml} \equiv \mu_{l} \cos b$.

After this, the program calculates $x$, $y$, and $z$ in galactic rectangular coordinates according to:

$$x = d \cos b \cos l$$

$$y = d \cos b \sin l$$

$$z = d \sin b$$

Where $d$ is the distance to the star. Taking the time derivative of these equations allows us to calculate $v_x$, $v_y$, and $v_z$:

$$v_x = v_r \cos b \cos l - d \mu_b \sin b \cos l - d \mu_l \cos b \sin l$$

$$v_y = v_r \cos b \sin l - d \mu_b \sin b \sin l + d \mu_l \cos b \cos l$$

$$v_z = v_r \sin b + d \mu_b \cos b$$

Where $d$ is the distance to the star. In the code, a conversion factor $k$ is used in the velocity equations to convert $\textrm{kpc mas/yr}$ to $\textrm{km/s}$. Once the units have been converted, the query checks for stars satisfying the equation:

$$\sqrt{(x-x_o)^2 + (y-y_0)^2 + (z-z_0)^2 + (A(v_x - v_{x_0}))^2 + (A(v_y - v_{y_0}))^2 + (A(v_z - v_{z_0}))^2} < \epsilon$$

Where $A$ is a scale factor that can be adjusted to weight the velocities similarly to the positions. Example usages are below.

In [1]:
from search_phase_space import *

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [2]:
help(search_phase_space)

Help on function search_phase_space in module search_phase_space:

search_phase_space(x, y, z, vx, vy, vz, epsilon, v_scale=1.0, cone_r=None)
    NAME:
        search_phase_space
        
    PURPOSE:
        query the Gaia DR2 RV catalogue for stars near a point in phase space
        
    INPUT:
        x - rectangular x coordinate in the galactic frame (can be Quantity,
        otherwise given in kpc)
        
        y - rectangular y coordinate in the galactic frame (can be Quantity,
        otherwise given in kpc)
        
        z - rectangular z coordinate in the galactic frame (can be Quantity,
        otherwise given in kpc)
        
        vx - x velocity in the galactic frame (can be Quantity, otherwise given
        in km/s)
        
        vy - y velocity in the galactic frame (can be Quantity, otherwise given
        in km/s)
        
        vz - z velocity in the galactic frame (can be Quantity, otherwise given
        in km/s)
        
        epsilon - radius in p

In [3]:
table = search_phase_space(1,1,1,1,1,1,10,cone_r=1.0)
table['x','y','z','vx','vy','vz']

Query finished.


x,y,z,vx,vy,vz
float64,float64,float64,float64,float64,float64
0.3248547285099525,0.3279740455677438,0.3153034498744913,-0.29035230958035463,-5.1796266147947,-4.353165870439843
0.2188858173997254,0.2178464318294424,0.22580041586685468,-1.275905453220787,3.428241380577102,-1.4298321419533973
0.12780404047322055,0.12814092607639407,0.13156138199108242,1.046825144670162,3.4067004099763505,-1.123290133914609
0.2843689575919812,0.2885781542557433,0.29174504457791295,-4.331893530634634,2.6449143076819963,6.045735212272312


We will get different results if we adjust v_scale or epsilon.

In [4]:
table = search_phase_space(1,1,1,1,1,1,10,cone_r=1.0,v_scale=0.01)
len(table)

Query finished.


300

In [5]:
table = search_phase_space(1,1,1,1,1,1,100,cone_r=1.0)
len(table)

Query finished.


241

We can also change the initial search radius. This will allow us to check more stars, but will slow down the query.

In [6]:
%time table0 = search_phase_space(1,1,1,1,1,1,10,cone_r=1.0)
%time table1 = search_phase_space(1,1,1,1,1,1,10,cone_r=5.0)
%time table2 = search_phase_space(1,1,1,1,1,1,10,cone_r=10.0)

Query finished.
Wall time: 6.48 s
Query finished.
Wall time: 8.28 s
Query finished.
Wall time: 19.1 s


In [7]:
len(table0), len(table1), len(table2)

(4, 68, 312)

In [ ]:
table2['x','y','z','vx','vy','vz']

x,y,z,vx,vy,vz
float64,float64,float64,float64,float64,float64
0.16351177301492636,0.20967086726815753,0.17170827057914312,5.007038168658844,-5.682057274359033,3.170360553207343
0.11991166392645726,0.09274697055003087,0.13076461265609785,-1.8656472920941969,0.6451364718064411,-1.4451070652991378
0.24538553972665322,0.17922668423598098,0.25854831313742505,-1.0488419344827302,7.722052385780204,1.5919957093000883
0.5862335720419103,0.4079657249550528,0.6177366918912254,-4.243596474243831,-4.491349178841477,-2.205186284387413
0.37299120418406456,0.26536909556021476,0.3897092992017556,3.8762811910623576,-5.643842192029282,-2.902144773623549
0.2635157495689372,0.18538711101100866,0.27676805528664905,8.647591791880037,0.8181813660317027,-4.068180081473987
0.1498358840130167,0.10869398590374628,0.16540172957859245,-4.31159345079178,1.2221947891316831,-5.390067092844093
0.05279017187520276,0.03886285924095198,0.05848680893002437,1.5123623387606568,-0.3313365259577292,-1.7068390104272426
0.14468619608274713,0.10633360032470704,0.16444671085387466,3.3903564922764433,8.404732465333476,4.3317073730660365


Scaling the velocities to be similar to the positions allows us to reduce the value of epsilon.

In [ ]:
table = search_phase_space(1,1,1,1,1,1,1,cone_r=10.0,v_scale=0.1)
table['x','y','z','vx','vy','vz']

### Update 2018-05-26
The function now performs a distance check to further limit the initial size of the query. If $d$ is the distance to the search point from the Sun and $p$ is the parallax of a star, then the query is now limited to stars where $\left|d-\frac{1}{p}\right|<\epsilon$. This limits the search to a spherical shell centred around the Sun with inner radius $d-\epsilon$ and outer radius $d+\epsilon$.

The behaviour of the cone search has also been modified. If cone_r is set to None, then the function will compute the minimal cone radius such that the entire physical search sphere is encompassed by the cone. The calculations for this are shown below.

Consider a 2D cross section of the search sphere centred at the origin, and suppose the Sun is at point $(d, 0)$. The upper portion of the cross section is described by $f(x)=\sqrt{(\epsilon^2-x^2)}$. Thus, we have $f'(x)=-\frac{x}{\sqrt{(\epsilon^2-x^2)}}$. The edge of the minimal cone will the be the tangent line of $f(x)$ that intersects $(d,0)$. It can be shown that the point on the circle where this occurs is at $x=\frac{\epsilon^2}{d}$. Setting $h=d-\frac{\epsilon^2}{d}$ and $r=f(\frac{\epsilon^2}{d})$, we have the radius of the cone in degrees is $\arctan(\frac{r}{h})$.

A visual demonstration of the new behaviour can be seen here: https://www.desmos.com/calculator/ezmhltuapi. The search area is the intersection of the red and blue areas.

If the Sun is contained in the search radius, then only the distance check will be used, and the cone search will be skipped. If cone_r is set manually, however, the cone search will be done no matter what.